In [ ]:
!pip install ipywidgets

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
!pip install -r requirements.txt


You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
import json
import pandas as pd
import numpy as np
import math as math
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.model_selection as model_selection
import sklearn.tree as tree
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
import time
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
import catboost
from catboost import CatBoostClassifier
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split


In [ ]:
df_train = pd.read_csv('Training_clean1.csv')
def root_mean_squared_log_error(y_true, y_pred):
    # Alternatively: sklearn.metrics.mean_squared_log_error(y_true, y_pred) ** 0.5
    assert (y_true >= 0).all() 
    assert (y_pred >= 0).all()
    log_error = np.log1p(y_pred) - np.log1p(y_true)  # Note: log1p(x) = log(1 + x)
    return np.mean(log_error ** 2) ** 0.5

df_test = pd.read_csv('Test_clean1.csv')
    


In [ ]:
df_train['floor/stories'] = df_train.floor/df_train.stories

In [ ]:

y1 = df_train['price']/df_train['area_total'].values
X1 = df_train.drop(['id','price', 'street', 'address', 'area_total'], axis=1)
X1_test = df_test.drop(['id', 'street', 'address', 'area_total'], axis=1)


In [ ]:
#y1 = df_train.price.values
#X1 = df_train.drop(['id','price', 'street', 'address'], axis=1)
#X1_test = df_test.drop(['id', 'street', 'address'], axis=1)
categorical_features_indices = np.where(X1.dtypes != np.float64)[0]
categorical_features_indices = np.where(X1_test.dtypes != np.float64)[0]


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X1, y1, test_size=0.2, random_state=1, shuffle=True)
X_train.shape, X_val.shape, y_train.shape,y_val.shape, X1_test.shape


((18628, 22), (4657, 22), (18628,), (4657,), (9937, 21))

In [ ]:
#Graid searchCV hyper parameter
n_estimators=[100,500,900,1100,1500]
max_depth=[2,3,5,10,15]
booster=["gbtree","gblinear"]
learning_rate="[0.05,0.1,0.15,0.20]"
min_child_weight=[1,2,3,4]
base_score=[0.25,0.5,0.75,1]

org_params = {
    'n_estimators': n_estimators,
    'learning_rate': learning_rate,
    "booster":booster,
    'eval_metric': 'RMSE',
    'use_best_model': True,
    'loss_function': 'RMSE',
    'max_depth': max_depth,
    "min_child_weight":min_child_weight,
    "base_score":base_score
}

In [ ]:
model_CBR = CatBoostRegressor()

In [ ]:
parameters = {'depth'         : [6,8,10],
                  'learning_rate' : [0.005, 0.01, 0.05],
                  'iterations'    : [1500,2500,3500,5000]
                 }

In [ ]:
grid = GridSearchCV(estimator=model_CBR, param_grid = parameters, cv = 5, n_jobs=-1)
grid.fit(X1, y1)
print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n", grid.best_estimator_)
print("\n The best score across ALL searched params:\n", grid.best_score_)
print("\n The best parameters across ALL searched params:\n", grid.best_params_)

In [ ]:

regressor = CatBoostRegressor(depth= 8, iterations= 5000, learning_rate= 0.05)

NameError: name 'CatBoostRegressor' is not defined

In [ ]:
#randon searchCV
import xgboost
regressor=xgboost.XGBRegressor()

from sklearn.model_selection import RandomizedSearchCV
booster=['gbtree','gblinear']
base_score=[0.25,0.5,0.75,1]
## Hyper Parameter Optimization

n_estimators = [900, 1300, 1600,2000]
max_depth = [7,8,9]
booster=['gbtree','gblinear']
learning_rate=[0.05,0.1,0.15]
min_child_weight=[1,2,3,4]

# Define the grid of hyperparameters to search
hyperparameter_grid = {
    'n_estimators': n_estimators,
    'max_depth':max_depth,
    'learning_rate':learning_rate,
    'min_child_weight':min_child_weight,
    'booster':booster,
    'base_score':base_score
    }

In [ ]:
# Set up the random search with 5-fold cross validation
random_cv = RandomizedSearchCV(estimator=regressor,
            param_distributions=hyperparameter_grid,
            cv=5, n_iter=50,
            scoring = 'neg_mean_absolute_error',n_jobs = 4,
            verbose = 5, 
            return_train_score = True,
            random_state=42)

In [ ]:
random_cv.fit(X1,y1)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


KeyboardInterrupt: 

In [ ]:
random_cv.best_estimator_

In [ ]:
from catboost import CatBoostRegressor
#random_state=1, iterations=1500, depth=8, learning_rate=.1, loss_function='RMSE'
model = CatBoostRegressor(random_state=1, iterations=2000, depth=8, learning_rate=0.1, loss_function='RMSE')
model.fit(X_train, y_train,cat_features=categorical_features_indices,eval_set=(X_val, y_val),plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

1005:	learn: 30295.9519351	test: 76687.5510987	best: 76684.2197308 (1004)	total: 1m 58s	remaining: 1m 57s
1006:	learn: 30276.1178974	test: 76692.1975483	best: 76684.2197308 (1004)	total: 1m 58s	remaining: 1m 57s
1007:	learn: 30259.1642865	test: 76683.9189217	best: 76683.9189217 (1007)	total: 1m 59s	remaining: 1m 57s
1008:	learn: 30233.6062014	test: 76695.1371029	best: 76683.9189217 (1007)	total: 1m 59s	remaining: 1m 56s
1009:	learn: 30208.6705671	test: 76681.5376482	best: 76681.5376482 (1009)	total: 1m 59s	remaining: 1m 56s
1010:	learn: 30176.1030959	test: 76658.6878426	best: 76658.6878426 (1010)	total: 1m 59s	remaining: 1m 56s
1011:	learn: 30172.1609082	test: 76659.4053793	best: 76658.6878426 (1010)	total: 1m 59s	remaining: 1m 56s
1012:	learn: 30171.5483933	test: 76659.3389909	best: 76658.6878426 (1010)	total: 1m 59s	remaining: 1m 56s
1013:	learn: 30154.0925239	test: 76654.1698254	best: 76654.1698254 (1013)	total: 1m 59s	remaining: 1m 56s
1014:	learn: 30132.8990937	test: 76649.2030003

In [ ]:
print(model.score(X_train, y_train))
from sklearn.metrics import r2_score

y_pred = model.predict(X_val)
y_pred = y_pred.astype(float)
print(model.score(X_val, y_val)), print(r2_score(y_pred, model.predict(X_val)))

0.9512563684369933
0.800986734024097
1.0


(None, None)

In [ ]:
df=pd.DataFrame({'Actual': y_val, 'Predicted':y_pred})
df

,Actual,Predicted
10307,641025.641026,610668.885476
2684,370277.777778,434309.844730
15931,213851.992410,183244.908960
10683,187598.116170,168366.712469
22217,189556.509299,453746.533418
...,...,...
5606,119453.752094,132800.903035
8243,500000.000000,864574.271580
13490,163109.756098,170856.129217
7008,179500.000000,167433.255404


In [ ]:
final_labels = model.predict(X1_test)
final_labels = final_labels.astype(float)
final_labels

array([486725.09990443, 166303.00596529, 162085.64090198, ...,
       227744.66705782, 255909.39672418, 100767.35696157])

In [ ]:

final_labels = final_labels * df_test.area_total.values
len(final_labels)

9937

In [ ]:
y_train_hat = model.predict(X_train) * X_train.area_total.values ## * edit
y_valid_hat = model.predict(X_val) * X_val.area_total.values ## * edit

y_train = y_train * df_train.area_total.values ## edit
y_valid = y_valid * df_valid.area_total.values ## edit

print(f'Train RMSLE: {root_mean_squared_log_error(y_true=y_train, y_pred=y_train_hat) :.4f}')
print(f'Valid RMSLE: {root_mean_squared_log_error(y_true=y_val, y_pred=y_valid_hat) :.4f}')

#Train RMSLE: 0.1283
#Valid RMSLE: 0.1761

ValueError: operands could not be broadcast together with shapes (18628,) (23285,) 

In [ ]:
submission = pd.DataFrame()
submission['id'] = df_test.id
submission['price_prediction'] = final_labels # Predict on non-nan entries
 # Fill missing entries with mean predictor
print(f'Generated {len(submission)} predictions')

# Export submission to csv with headers
#submission.to_csv('sample_submission.csv', index=False)

submission.to_csv('sample_submission8.csv', index=False)

Generated 9937 predictions


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=99c1e030-46bc-40fa-835c-0661eae0d488' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>